OSIsoft Cloud Services REST API

requirements:<br>
Juptyer notebook or lab<br>
Python ocs_sample_library https://github.com/osisoft/OSI-Samples/tree/master/ocs_samples/library_samples/Python3<br>
config.ini configured for your OSIsoft Cloud Services environment https://github.com/osisoft/OSI-Samples/tree/master/ocs_samples/basic_samples/SDS/Python/SDSPy/Python3<br>

objective:<br>
Create metadata and tags for streams

In [1]:
# specify a unique prefix to create objects, suggestion use your nickname
example_prefix = 'SDSPrefix.'
# setup the environment, using statements defined in a python script file
%run setup.py

In [2]:
# verify we have a connection to the tenant, the tenant name should be returned
ocsClient.tenant

'efe27258-f6d5-4ea6-a001-3e4a82777710'

In [3]:
# list the streams - if these do not exist, run the previous notebook to create
for stream in ocsClient.Streams.getStreams(namespace_id,query=f'{example_prefix}Bearing*'):
    print(f'name: {stream.Name}, id: {stream.Id}')

name: bearing3, id: SDSPrefix.Bearing3
name: bearing2, id: SDSPrefix.Bearing2
name: bearing1, id: SDSPrefix.Bearing1
name: bearing4, id: SDSPrefix.Bearing4


In [4]:
# define a helper function to update tags for stream(s)

#
#add_tags - add a tag to supplied streams. If reset is True clear any existing tags first
#
def add_tags(namespace_id,streams,tag,reset=False):
    for stream in streams:
        # Check if there are any existing tags and if so add the new tag
        tags = ocsClient.Streams.getTags(namespace_id,stream.Id)
        if tags is not None and not reset:
            if tag not in tags:
                tags.append(tag)
        else:
            tags = [tag]
        ocsClient.Streams.createOrUpdateTags(namespace_id,stream.Id,tags)

In [5]:
# cooler sensor streams
tag = "sensor_cooler"
streams = ocsClient.Streams.getStreams(namespace_id,query="{prefix}Bearing1 \
                                              or {prefix}Bearing2".format(prefix=example_prefix))
add_tags(namespace_id,streams,tag,reset=True) 

# hydraulic sensor streams
tag = "sensor_hydraulic"
streams = ocsClient.Streams.getStreams(namespace_id,query="{prefix}Bearing3 \
                                              or {prefix}Bearing4".format(prefix=example_prefix))

add_tags(namespace_id,streams,tag,reset=True)

# sensor streams
tag = "sensor"
streams = ocsClient.Streams.getStreams(namespace_id,query='{prefix}Bearing*'.format(prefix=example_prefix))               
add_tags(namespace_id,streams,tag,reset=False)

In [6]:
# query streams using tag name
sorted([(stream.Name,stream.Id) for stream in ocsClient.Streams.getStreams(namespace_id,query='sensor_cooler')])
sorted([(stream.Name,stream.Id) for stream in ocsClient.Streams.getStreams(namespace_id,query='sensor_hydraulic')])
sorted([(stream.Name,stream.Id) for stream in ocsClient.Streams.getStreams(namespace_id,query='sensor')])

[('bearing1', 'SDSPrefix.Bearing1'), ('bearing2', 'SDSPrefix.Bearing2')]

[('bearing3', 'SDSPrefix.Bearing3'), ('bearing4', 'SDSPrefix.Bearing4')]

[('bearing1', 'SDSPrefix.Bearing1'),
 ('bearing2', 'SDSPrefix.Bearing2'),
 ('bearing3', 'SDSPrefix.Bearing3'),
 ('bearing4', 'SDSPrefix.Bearing4')]

In [7]:
# create metadata entry for each stream
for stream in ocsClient.Streams.getStreams(namespace_id,query='sensor'):
    ocsClient.Streams.createOrUpdateMetadata(namespace_id,stream.Id,{"site":"Palo Verde"})

In [8]:
# return the value for the metadata key: site 
for stream in ocsClient.Streams.getStreams(namespace_id,query='sensor'):
    print(f'{stream.Name}: {ocsClient.Streams.getMetadata(namespace_id,stream.Id,"site")}')

bearing3: Palo Verde
bearing2: Palo Verde
bearing1: Palo Verde
bearing4: Palo Verde


In [ ]:
# Clean-up environment - delete type and streams with {example_prefix} prefix defined earlier

def cleanup(run=False):
        for stream in ocsClient.Streams.getStreams(namespace_id,f'{example_prefix}*'):
            print(f'stream id: {stream.Id}, stream name: {stream.Name}')
            if run:
                try:
                    ocsClient.Streams.deleteStream(namespace_id,stream.Id)
                except:
                    pass
        for type in ocsClient.Types.getTypes(namespace_id,query=f'{example_prefix}*'):
            print(f'type id: {type.Id}, type name: {type.Name}')
            if run:
                try:
                    ocsClient.Types.deleteType(namespace_id,type.Id)
                    ocsClient.Types.deleteType(namespace_id,type.TypeId)
                except:
                    pass
        
# cleanup - disabled, i.e: False and commented:
#cleanup(False)